In [1]:
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

from read_dataset import read_ceps1d_with_train_test
from read_saved_models import loadMfcc1dStanderdScaler

Using TensorFlow backend.


In [2]:
def getStanderizedData(data):
    data_shape = data.shape
    n = data_shape[0]
    reshaped_data = data.reshape(n, -1)
    saved_ss = loadMfcc1dStanderdScaler()
    trasformed_data = saved_ss.transform(reshaped_data)
    ret_data = trasformed_data.reshape(data_shape)
    return ret_data

In [3]:
X_train, X_test, y_train, y_test = read_ceps1d_with_train_test()
X_ss_train = getStanderizedData(X_train)
X_ss_test = getStanderizedData(X_test)

In [4]:
seq = Sequential()
seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, 1, 30, 1293),
                   padding='same', return_sequences=True, activation='relu', data_format='channels_first'))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=1, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='binary_crossentropy', optimizer='adadelta')


In [5]:
result = seq.fit(X_ss_train, y_train, batch_size=10,
        epochs=300, validation_split=0.05)

ValueError: Error when checking input: expected conv_lst_m2d_1_input to have 5 dimensions, but got array with shape (600, 1, 30, 1293)